In [ ]:
import time
notebookstart= time.time()

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import pickle
# print("Data:\n",os.listdir("../data"))

In [4]:
print("\nData Load Stage")
training = pd.read_csv('../data/train.csv', index_col = "item_id", parse_dates = ["activation_date"])
traindex = training.index
testing = pd.read_csv('../data/test.csv', index_col = "item_id", parse_dates = ["activation_date"])
testdex = testing.index


Data Load Stage


# Extract the previous deal prob
### Here, we just extract part of it (for it's too large, we can do it separately for several times). This is used for train the model to understand this column

In [7]:
training_9_12 = training[training.isin(training.user_id.value_counts()[90000:120000].index)]

In [ ]:
from tqdm import tqdm

for i in tqdm(training.user_id.value_counts()[90000:120000].index):
    df_user = training_9_12.query("user_id == @i")
    training['prev_deal_prob'] = df_user.sort_values('item_seq_number').deal_probability.shift()

# For test data

# Create user's mean-value-deal-prob. if they have more 1 transaction

#### In training


- 771769 unique users
- 240019 repeated users

#### test set
- 306069 unique users
- 75237 repeated users (within test set)

#### overlapping ads in test set with previous repeated users:
- 105114

In [5]:
(training.user_id.value_counts() > 1).sum()

240019

In [6]:
(testing.user_id.value_counts() > 1).sum()

75237

In [20]:
repeat_users = training.user_id.value_counts()[:240019].index

In [24]:
repeat_users_deal_prob = training[training.user_id.isin(repeat_users)].groupby('user_id')['deal_probability'].mean()

In [29]:
df_repeat_users_deal_prob = repeat_users_deal_prob.to_frame().reset_index()

In [31]:
df_repeat_users_deal_prob.columns = ['user_id', 'prev_deal_prob']

In [33]:
df_repeat_users_deal_prob.head()

,user_id,prev_deal_prob
0,0000326c6042,0.000000
1,0000ae7a109b,0.000000
2,0000ba56add7,0.092233
3,000178adae29,0.062895
4,000185c6e01e,0.167210


In [38]:
df_repeat_users_deal_prob.head()

,user_id,prev_deal_prob
0,0000326c6042,0.000000
1,0000ae7a109b,0.000000
2,0000ba56add7,0.092233
3,000178adae29,0.062895
4,000185c6e01e,0.167210


In [43]:
testing.shape

(508438, 16)

In [39]:
testing_prev_deal_prob = pd.merge(testing, df_repeat_users_deal_prob, how='left', on='user_id').set_index(testing.index)

In [44]:
testing_prev_deal_prob.prev_deal_prob.describe()

count    105114.000000
mean          0.102210
std           0.157475
min           0.000000
25%           0.000000
50%           0.039782
75%           0.127059
max           1.000000
Name: prev_deal_prob, dtype: float64

In [48]:
# extract only prev_deal_prob this column

testing_prev_deal_prob = testing_prev_deal_prob.prev_deal_prob

In [51]:
testing_prev_deal_prob.to_csv('../data/testing_prev_deal_prob.csv')